In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


학습 데이터를 만든다.

In [2]:
# xData => [공부시간, 과외시간]
xData = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]] # 7행 2열

In [3]:
# yData => [합격여부] => 실제값
temp = [0, 0, 0, 1, 1, 1, 1] # 1행 7열

yData = np.array(temp)
print(yData)
print(type(yData))

# reshape(): numpy에서 데이터는 그대로 유지한채 배열의 형태(차원)을 변경한다.
# 1행 7열 numpy 배열 => 7행 1열 numpy 배열
yData = np.array(temp).reshape(7, 1)

[0 0 0 1 1 1 1]
<class 'numpy.ndarray'>


xData와 yData를 저장할 placeholder를 만든다.

In [4]:
# placeholder에 numpy 배열을 대입하는 경우 shape 속성을 이용해서 대입되는 numpy 배열의 차원을 지정해야 한다.
# [none, 2] => placeholder에 대입되는 numpy 배열의 행의 개수는 상관 없고, 열의 개수는 2개
X = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # yData가 대입될 placeholder

가중치, a, y절편(b)를 임의의 값으로 정한다.

In [5]:
a = tf.Variable(tf.random_uniform([2, 1]), dtype=tf.float32) # 난수를 2행 1열로 발생시킨다.
b = tf.Variable(tf.random_uniform([1]), dtype=tf.float32)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a1: {}, a2: {}, b: {}'.format(sess.run(a[0][0]), sess.run(a[1][0]), sess.run(b)))
print('a\n', sess.run(a), sep='')
print('b\n', sess.run(b), sep='')

a1: 0.1390218734741211, a2: 0.4984704256057739, b: [0.8316431]
a
[[0.13902187]
 [0.49847043]]
b
[0.8316431]


시그모이드 방정식, 오차 함수, 경사 하강법

In [9]:
# tf.sigmoid(): 시그모이드 방정식을 계산한다.
# tf.matmul(): 행렬의 곱(적)을 계산한다.
y = tf.sigmoid(tf.matmul(X, a) + b) # 시그모이드 방정식
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y)) # 오차 함수
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss) # 경사 하강법

sigmoid() 함수의 실행 결과(y)가 0.5 이상이면 1을 0.5 미만이면 -1을 리턴시킨다.

In [11]:
sess = tf.Session()
# cast(x, dtype): x에 저장된 데이터를 dtype으로 형변환한다.
predict = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32)
print(sess.run(predict)) # [1 2]
# cast() 함수는 캐스팅할 데이터가 boolean 타입이면 True는 1로 False는 0으로 형변환한다.
predict = tf.cast([True, False], dtype=tf.int32)
print(sess.run(predict)) # [1 0]
predict = tf.cast([0.4 >= 0.5, 0.5 >= 0.5], dtype=tf.int32)
print(sess.run(predict))

[1 2]
[1 0]
[0 1]


In [16]:
predict = tf.cast(y >= 0.5, dtype=tf.float32) # 예측값, sigmoid() 함수를 실행한 결과를 0 또는 1로 변환한다.

예측값(predict)과 실제값(Y)이 일치하는 정도(정확도, accuracy)를 계산한다.

In [17]:
sess = tf.Session()
# equal(): 인수로 지정한 값이 같으면 True, 다르면 False를 리턴한다.
# print(sess.run(tf.equal(1, 1)), sess.run(tf.equal(1, 0))) # True False
# equal() 함수로 예측값과 실제값이 같은가 비교 후 True 또는 False를 1또는 0으로 캐스팅하고 전체 평균을 계산한다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.int32))

In [23]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(60001):
    # sess.run([실행할 내용, ...], feed_dict={X: 입력 데이터, Y: 입력 데이터에 따른 실제값})
    a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict={X: xData, Y: yData})
    if epoch % 3000 == 0:
        print('epoch: {:4d}, a1: {:9.5f}, a2: {:9.5f}, b: {:9.5f}, loss: {:9.5f}'.format(epoch, a_[0][0], a_[1][0], b_[0], loss_))

epoch:    0, a1:   0.55070, a2:   0.03570, b:   0.43736, loss:   1.70046
epoch: 3000, a1:   0.18269, a2:   1.55716, b:  -8.84457, loss:   0.05815
epoch: 6000, a1:  -0.16015, a2:   2.55288, b: -11.46681, loss:   0.03058
epoch: 9000, a1:  -0.33368, a2:   3.11215, b: -13.06593, loss:   0.02079
epoch: 12000, a1:  -0.44770, a2:   3.50066, b: -14.21992, loss:   0.01577
epoch: 15000, a1:  -0.53241, a2:   3.79879, b: -15.12364, loss:   0.01272
epoch: 18000, a1:  -0.59979, a2:   4.04090, b: -15.86676, loss:   0.01066
epoch: 21000, a1:  -0.65572, a2:   4.24481, b: -16.49800, loss:   0.00918
epoch: 24000, a1:  -0.70354, a2:   4.42102, b: -17.04672, loss:   0.00806
epoch: 27000, a1:  -0.74531, a2:   4.57620, b: -17.53217, loss:   0.00719
epoch: 30000, a1:  -0.78240, a2:   4.71487, b: -17.96750, loss:   0.00649
epoch: 33000, a1:  -0.81575, a2:   4.84021, b: -18.36209, loss:   0.00591
epoch: 36000, a1:  -0.84602, a2:   4.95455, b: -18.72309, loss:   0.00543
epoch: 39000, a1:  -0.87378, a2:   5.05970

테스트

In [51]:
# 테스트 데이터를 만든다.
new_x = np.array([6, 5]).reshape(1, 2) # 테스트 데이터가 1차원 numpy 배열로 만들어지기 때문에 행렬의 곱셈 연산이 실행되지 않는다. => reshape()

# sigmoid() 함수의 연산 결과에 테스트 데이터를 대입해서 연산한다.
result, new_y = sess.run([predict, y], feed_dict={X: new_x})
print('공부시간: {}, 과외시간: {}'.format(new_x[:, 0][0], new_x[:, 1][0]))
print('합격여부: {}'.format('합격' if result[0][0] == 1 else '불합격'))
print('합격확률: {}%'.format(new_y * 100))

공부시간: 6, 과외시간: 5
합격여부: 합격
합격확률: [[75.83376]]%


In [53]:
for i in range(23):
    
    # 테스트 데이터를 만든다.
    new_x = np.array([0, i]).reshape(1, 2) # 테스트 데이터가 1차원 numpy 배열로 만들어지기 때문에 행렬의 곱셈 연산이 실행되지 않는다. => reshape()

    # sigmoid() 함수의 연산 결과에 테스트 데이터를 대입해서 연산한다.
    result, new_y = sess.run([predict, y], feed_dict={X: new_x})
    print('공부시간: {}, 과외시간: {}'.format(new_x[:, 0][0], new_x[:, 1][0]))
    print('합격여부: {}'.format('합격' if result[0][0] == 1 else '불합격'))
    print('합격확률: {}%'.format(new_y * 100))
    if result[0][0] == 1:
        break

공부시간: 0, 과외시간: 0
합격여부: 불합격
합격확률: [[8.781558e-08]]%
공부시간: 0, 과외시간: 1
합격여부: 불합격
합격확률: [[2.4353656e-05]]%
공부시간: 0, 과외시간: 2
합격여부: 불합격
합격확률: [[0.00675394]]%
공부시간: 0, 과외시간: 3
합격여부: 불합격
합격확률: [[1.8386185]]%
공부시간: 0, 과외시간: 4
합격여부: 합격
합격확률: [[83.85662]]%
